# Catalogue_v1_5

Catalogue files in a directory: file type and size. 

V1.4: include python-magic to extract filetype from file
V1.5: run via docker, linux file system paths

In [20]:
# import pip #needed to use the pip functions
# for i in pip.get_installed_distributions(local_only=True):
#    print(i)

In [1]:
import magic

# file_magic = magic.Magic(magic_file="C:\\Users\\rothw\\Anaconda3\\magic.mgc")

ModuleNotFoundError: No module named 'magic'

In [9]:
# Imports
import os, sys
from os import listdir, environ
from os.path import isfile, join

import pandas as pd

In [10]:
# Get current working directory

os.getcwd()

'/opt/notebooks'

In [23]:
# Set directory

### Newdata Laptop
# my_path = '/opt/paper/Op_Uucan_Cleansed/Items'

### Windows Laptop
# my_path = 'C:\\Users\\rothw\\Documents\\BDEC\\Data\\enron-test-data'
my_path = '/opt/mail/test-data/Archive'

print(my_path)

# os.listdir(my_path)

/opt/mail/test-data/Archive


['11.',
 '13.',
 '2.',
 '1.',
 '9.',
 '7.',
 '8.',
 '12.',
 '4.',
 '3.',
 '10.',
 '5.',
 '6.']

In [12]:
# Test on one file

### Newdata Laptop
# bb='[Unnamed Image]_3405.jpeg'

### Windows laptop
bb = '1.'

print("File size = " + str(os.path.getsize(join(my_path, bb))))

File size = 1126


##### filetype example - now uninstalled as didn;t support filetypes of interest, such as doc, etc.

print ("File type = ")
kind = filetype.guess(join(my_path, bb))
if kind == None:
    print('None')
else:
    print(kind.extension)
    print(kind.mime)

In [13]:
magic_object = magic.Magic(magic_file="C:\\Users\\rothw\\Anaconda3\\magic.mgc", mime=True)

# help(magic_object)

magic_object.from_file(join(my_path, bb))

NameError: name 'magic' is not defined

In [15]:
# Loop through all files in directory

allfiles=[]
urn = 0

for subdir, dirs, files in os.walk(my_path):
    for file in files:
        urn += 1
        row=[]
        row.append(subdir)
        row.append(file)
#        if isfile(join(my_path, file)):
        row.append(os.path.getsize(join(subdir, file)))
        row.append(os.path.splitext(join(subdir, file))[1].upper())
###        row.append(magic_object.from_file(join(subdir, file)))
        print(row)
        allfiles.append(row)

['/opt/mail/test-data/Archive', '11.', 480, '.']
['/opt/mail/test-data/Archive', '13.', 974, '.']
['/opt/mail/test-data/Archive', '2.', 1405, '.']
['/opt/mail/test-data/Archive', '1.', 1126, '.']
['/opt/mail/test-data/Archive', '9.', 649, '.']
['/opt/mail/test-data/Archive', '7.', 1199, '.']
['/opt/mail/test-data/Archive', '8.', 586, '.']
['/opt/mail/test-data/Archive', '12.', 884, '.']
['/opt/mail/test-data/Archive', '4.', 870, '.']
['/opt/mail/test-data/Archive', '3.', 1522, '.']
['/opt/mail/test-data/Archive', '10.', 478, '.']
['/opt/mail/test-data/Archive', '5.', 798, '.']
['/opt/mail/test-data/Archive', '6.', 11895, '.']


In [16]:
type(allfiles)

list

In [17]:
len(allfiles)

13

In [18]:
urn

13

In [19]:
# Convert to DataFrame
allfiles_df = pd.DataFrame(allfiles)

# Rename columns
allfiles_df.columns = ['subdir', 'filename', 'size', 'fileext', 'mimetype']

# Split out mimetype
allfiles_df[['mime1', 'mime2']] = pd.DataFrame(allfiles_df.mimetype.str.split('/').tolist(), columns=['mime1', 'mime2'])

allfiles_df.head(30)

ValueError: Length mismatch: Expected axis has 4 elements, new values have 5 elements

In [ ]:
# Output to CSV

allfiles_df.to_csv('/opt/outputs/results.csv')

In [60]:
# Summarise no. records by filetype

summary_count = allfiles_df[['filename', 'fileext']].groupby(['fileext']).count()

filecount_total = summary_count.sum()
print("filecount_total = " + str(filecount_total))

summary_count['percent'] = summary_count['filename'] / float(filecount_total)

summary_count.rename(columns={'filename': 'num_files', 'percent': 'perc_num_files'}, inplace=True)

summary_count.sort_values('num_files', ascending=False).head(30)

filecount_total = filename    8467
dtype: int64


,num_files,perc_num_files
fileext,,
,8401,0.992205
.JPG,37,0.004370
.PDF,17,0.002008
.TXT,5,0.000591
.PNG,3,0.000354
.DS_STORE,2,0.000236
.DOCX,1,0.000118
.ZIP,1,0.000118


In [61]:
# Summarise size by filetype

summary_size = allfiles_df[['size', 'fileext']].groupby(['fileext']).sum()

size_total = summary_size.sum()
print("size_total = " + str(size_total))


summary_size['percent'] = summary_size['size'] / float(size_total)

summary_size.rename(columns={'percent': 'perc_size'}, inplace=True)

summary_size.sort_values('size', ascending=False).head(30)

size_total = size    33145118
dtype: int64


,size,perc_size
fileext,,
,17048698,0.514365
.ZIP,12485691,0.376698
.JPG,1306337,0.039413
.PDF,1286212,0.038805
.PNG,995603,0.030038
.DOCX,18236,0.000550
.TXT,4101,0.000124
.DS_STORE,240,0.000007


In [62]:
# Merge together num_files and size

summary_fileext = pd.merge(summary_count, summary_size, left_index=True, right_index=True)

summary_fileext

,num_files,perc_num_files,size,perc_size
fileext,,,,
,8401,0.992205,17048698,0.514365
.DOCX,1,0.000118,18236,0.000550
.DS_STORE,2,0.000236,240,0.000007
.JPG,37,0.004370,1306337,0.039413
.PDF,17,0.002008,1286212,0.038805
.PNG,3,0.000354,995603,0.030038
.TXT,5,0.000591,4101,0.000124
.ZIP,1,0.000118,12485691,0.376698


In [96]:
# Summarise by mimetype

# Number of files
summary_count_mime = allfiles_df[['filename', 'mimetype', 'mime1', 'mime2']].groupby(['mimetype', 'mime1', 'mime2']).count()
filecount_total_mime = summary_count_mime.sum()
summary_count_mime['perc_num_files'] = summary_count_mime['filename'] / float(filecount_total_mime)
summary_count_mime.rename(columns={'filename': 'num_files'}, inplace=True)

# File size
summary_size_mime = allfiles_df[['size', 'mimetype', 'mime1', 'mime2']].groupby(['mimetype', 'mime1', 'mime2']).sum()
size_total_mime = summary_size_mime.sum()
summary_size_mime['perc_size'] = summary_size_mime['size'] / float(size_total_mime)

# Merge num_files and size summaries together
summary_mimetype = pd.merge(summary_count_mime, summary_size_mime, left_index=True, right_index=True)

summary_mimetype

,,,num_files,perc_num_files,size,perc_size
mimetype,mime1,mime2,,,,
application/octet-stream,application,octet-stream,465,0.054919,139028,0.004195
application/pdf,application,pdf,17,0.002008,1286212,0.038805
application/vnd.openxmlformats-officedocument.wordprocessingml.document,application,vnd.openxmlformats-officedocument.wordprocessingml.document,1,0.000118,18236,0.000550
application/zip,application,zip,1,0.000118,12485691,0.376698
image/jpeg,image,jpeg,37,0.004370,1306337,0.039413
image/png,image,png,3,0.000354,995603,0.030038
text/html,text,html,21,0.002480,211343,0.006376
text/plain,text,plain,7887,0.931499,16395068,0.494645
text/x-c,text,x-c,1,0.000118,20339,0.000614


In [99]:
# Check percentage calcs - should add up to 1

print(summary_fileext.perc_num_files.sum())

print(summary_mimetype.perc_size.sum())

1.0000000000000002
1.0000000000000002


In [ ]:
# Export

# summary_count.to_csv('/opt/outputs/summary_count.csv')

# summary_size.to_csv('/opt/outputs/summary_size.csv')

summary_fileext.to_csv('/opt/outputs/summary_fileext.csv')

summary_mimetype.to_csv('/opt/outputs/summary_mimetype.csv')